# Analysis of the CID pinging phase

  

In [1]:
# Import dependencies
import sqlalchemy as sa
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## DB Credentials
HOST="localhost"
PORT="5432"
DB="hoarder_test"
USER="hoarder"
PASSWD="password"

# Connecte with the DB
engine = sa.create_engine(f'postgresql://{USER}:{PASSWD}@{HOST}:{PORT}/{DB}')

## plotting style
fig_size= (7,4)
sns.set_context("talk", font_scale=1)


In [2]:
## get the median time of each fetch time

sql_query="""
    SELECT 
        cid_hash,
        ping_round, 
        fetch_time
    FROM fetch_results
    ORDER BY ping_round;
"""
ping_rounds = pd.read_sql_query(sql_query, engine)

avg_fetcht = ping_rounds.groupby("ping_round").mean()
hours_dist = avg_fetcht["fetch_time"].to_numpy()

hours_dist = (hours_dist - hours_dist[0]) / 3600
print(hours_dist)

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Track the Activity or Onliness of those PR Holders
We divide them into:
1. Total PR Holders
2. Only non-hydra PR Holders
3. Only hydra PR Holders

In [ ]:
def plot_ping_dist(pd_obj, column_name, opts):
    ## Get the total active peers distribution per ping_round
    pv_table = pd_obj.pivot(index=["ping_round", "cid_hash"], columns=column_name, values="count")
    pv_table = pv_table.fillna(0)
    aux = pd.DataFrame(pv_table.to_records())

    # make dist
    dist = []
    for i, h in enumerate(hours_dist):
        t = aux.query(f"ping_round == {i}")
        dist.append(t["True"])
        
    ## Make a boxplot with the distribution
    fig, ax = plt.subplots(figsize=(12,6))
    ax.boxplot(dist, positions=hours_dist, showfliers=True) 
    ticks = np.linspace(0, 36, 10) ###### <---- *UPDATE THIS* to fit the study duration
    plt.xticks(ticks, ticks.astype(int))
    plt.xlabel("Time Since Publication (Hours)")
    plt.ylabel(opts["ylabel"])
    plt.show()

In [ ]:
## Get the active peers distribution per ping_round

sql_query = """
SELECT 
	ping.cid_hash,
	ping.ping_round,
	ping.is_active,
	count(ping.is_active)
FROM (
	SELECT 
		pr.cid_hash,
		pr.ping_round,
		pr.is_active, 
		pr.has_records,
		peer_info.client
	FROM ping_results as pr
	LEFT JOIN peer_info ON pr.peer_id = peer_info.peer_id
	ORDER BY ping_round ASC
) as ping
GROUP BY cid_hash, ping_round, is_active;
"""

ping_rounds = pd.read_sql_query(sql_query, engine)
print("")
plot_ping_dist(ping_rounds, "is_active", {"ylabel":"Peers Online"})

In [ ]:
## Get the active Non-Hydras PR Holders distribution per ping_round

sql_query = """
	SELECT 
		ping.cid_hash,
		ping.ping_round,
		ping.is_active,
		count(ping.is_active)
	FROM (
		SELECT 
			pr.cid_hash,
			pr.ping_round,
			pr.is_active, 
			pr.has_records,
			peer_info.client
		FROM ping_results as pr
		LEFT JOIN peer_info ON pr.peer_id = peer_info.peer_id
		ORDER BY ping_round ASC
	) as ping
	WHERE ping.client!='hydra-booster'
	GROUP BY cid_hash, ping_round, is_active;
"""

pings = pd.read_sql_query(sql_query, engine)
plot_ping_dist(pings, "is_active", {"ylabel":"Peers Online"})

In [ ]:
## Get the active Hydras PR Holders distribution per ping_round

sql_query = """
	SELECT 
		ping.cid_hash,
		ping.ping_round,
		ping.is_active,
		count(ping.is_active)
	FROM (
		SELECT 
			pr.cid_hash,
			pr.ping_round,
			pr.is_active, 
			pr.has_records,
			peer_info.client
		FROM ping_results as pr
		LEFT JOIN peer_info ON pr.peer_id = peer_info.peer_id
		ORDER BY ping_round ASC
	) as ping
	WHERE ping.client='hydra-booster'
	GROUP BY cid_hash, ping_round, is_active;
"""

pings = pd.read_sql_query(sql_query, engine)
plot_ping_dist(pings, "is_active", {"ylabel": "Peers Online"})

### Track the whether the PR Holders share the PRs
We divide them into:
1. Total PR Holders sharing the PRs
2. Only non-hydra PR Holders sharing the PRs
3. Only hydra PR Holders sharing the PRs

In [ ]:
## Get the distribution of the PR Holders that share the PRs per ping_round

sql_query = """
SELECT 
	ping.cid_hash,
	ping.ping_round,
	ping.has_records,
	count(ping.has_records)
FROM (
	SELECT 
		pr.cid_hash,
		pr.ping_round,
		pr.is_active, 
		pr.has_records,
		peer_info.client
	FROM ping_results as pr
	LEFT JOIN peer_info ON pr.peer_id = peer_info.peer_id
	ORDER BY ping_round ASC
) as ping
GROUP BY cid_hash, ping_round, has_records;
"""

pings = pd.read_sql_query(sql_query, engine)
plot_ping_dist(pings, "has_records", {"ylabel": "Peers Sharing PRs"})

In [ ]:
## Get the non-hydra PR Holders sharing the PRs per ping_round

sql_query = """
SELECT 
	ping.cid_hash,
	ping.ping_round,
	ping.has_records,
	count(ping.has_records)
FROM (
	SELECT 
		pr.cid_hash,
		pr.ping_round,
		pr.is_active, 
		pr.has_records,
		peer_info.client
	FROM ping_results as pr
	LEFT JOIN peer_info ON pr.peer_id = peer_info.peer_id
	ORDER BY ping_round ASC
) as ping
WHERE ping.client!='hydra-booster'
GROUP BY cid_hash, ping_round, has_records;
"""

pings = pd.read_sql_query(sql_query, engine)
plot_ping_dist(pings, "has_records", {"ylabel": "Peers Sharing PRs"})

In [ ]:
## Get the distribution of hydra peers sharing the PRs per ping_round

sql_query = """
SELECT 
	ping.cid_hash,
	ping.ping_round,
	ping.has_records,
	count(ping.has_records)
FROM (
	SELECT 
		pr.cid_hash,
		pr.ping_round,
		pr.is_active, 
		pr.has_records,
		peer_info.client
	FROM ping_results as pr
	LEFT JOIN peer_info ON pr.peer_id = peer_info.peer_id
	ORDER BY ping_round ASC
) as ping
WHERE ping.client='hydra-booster'
GROUP BY cid_hash, ping_round, has_records;
"""

pings = pd.read_sql_query(sql_query, engine)
plot_ping_dist(pings, "has_records", {"ylabel": "Peers Sharing PRs"})

In [ ]:
engine.dispose()